In [19]:
from collections import Counter 
import re
from dotenv import load_dotenv
import os
import json
# imports

from dotenv import load_dotenv
import os
import json
from openai import OpenAI #older code removed
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import chromadb
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
load_dotenv()
top_k = 4

In [20]:
# NEW - Use `openai.ChatCompletion.create()` format for `openai>=1.0.0`
@retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(3))
def completion_with_backoff(**kwargs):
    #client = openai.OpenAI()  # Create a client instance
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )

    return client.chat.completions.create(**kwargs)  # Updated API Call

In [21]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="text-embedding-ada-002"
)

In [22]:
#chroma db client: # database_name = "API_misuse_fix_pattern_rules"
# Define persist directory
persist_dir = "/home/akazad/test_tool_perfapim/llmapitool/my_tool_exp/chroma_db"

# Create directory if it doesn't exist
os.makedirs(persist_dir, exist_ok=True)
# after

client = chromadb.PersistentClient(path=persist_dir)

In [ ]:
# #DB

# #chroma db client
# # Define persist directory
# persist_dir = "/home/akazad/LLMAPIDet/exp_one_azad_cpp/chroma_db"

# # Create directory if it doesn't exist
# os.makedirs(persist_dir, exist_ok=True)
# # after

# client = chromadb.PersistentClient(path=persist_dir)

In [23]:
database_name = "cpp_fix_rules_stratified_ctxt3f5_enbedding"  # please refer to 2_chroma_DB.ipynb cell 5 for the database name
collection = client.get_or_create_collection(
    database_name, embedding_function=openai_ef)
collection_list = client.list_collections()
print(collection_list)

[Collection(name=cpp_fix_rules_stratified_t3f1_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f5_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f3_enbedding), Collection(name=cpp_fix_rules_s3_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt3f1_enbedding), Collection(name=cpp_fix_rules_stratified_t3f5_enbedding), Collection(name=cpp_fix_rules_stratified_t3f2_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt2f2_enbedding), Collection(name=cpp_fix_rules_v1_enbedding_975), Collection(name=cpp_fix_rules_stratified_ctxt2f1_enbedding), Collection(name=cpp_fix_rules_stratified_ctxt4f1_enbedding), Collection(name=cpp_fix_rules_s3_t2_enbedding), Collection(name=cpp_fix_rules_stratified_f2_enbedding), Collection(name=cpp_fix_rules_stratified_ctxf1_enbedding), Collection(name=cpp_fix_rules_stratified_f1_enbedding), Collection(name=cpp_fix_rules_stratified_t3f3_enbedding), Collection(name=cpp_fix_rules_stratified_t2f1_enbedding), Collection(name=cpp_fix_rules_

In [24]:
import re
def get_APIs_cpp(change):
    """
    Extract API call names from a C++ diff.
    It focuses only on added or deleted lines (those beginning with '+' or '-').
    It captures:
      1. Fully qualified calls, e.g. boost::filesystem::move(
         → returns "::move(" (the last component, prepended by "::")
      2. Dot-based calls, e.g. vec.push_back(
         → returns ".push_back(" (the operator and method)
      3. Arrow-based calls, e.g. ptr->get(
         → returns "->get(" (the operator and method)
      4. Unqualified function calls, e.g. abs(
         → returns "abs(" (provided the function name starts with a letter,
            and is not one of the control keywords)
    """
    API_set = set()
    
    # Pattern for fully qualified calls: capture the last function name after '::'
    pattern_fq = re.compile(r'\b(?:[A-Za-z_]\w*::)+([A-Za-z_]\w*)\s*\(')
    # Pattern for dot-based calls: capture the dot and function name (e.g., ".push_back")
    pattern_dot = re.compile(r'\b[A-Za-z_]\w*(\.[A-Za-z_]\w*)\s*\(')
    # Pattern for arrow-based calls: capture the arrow and function name (e.g., "->get")
    pattern_arrow = re.compile(r'\b[A-Za-z_]\w*(->[A-Za-z_]\w*)\s*\(')
    # Pattern for unqualified calls: ensure not preceded by a dot, colon, or arrow;
    # exclude keywords such as if, for, while, etc.
    pattern_unq = re.compile(
        r'(?<![\.:])(?<!->)\b(?!(if|for|while|switch|return|sizeof|catch|throw)\b)([A-Za-z]\w*)\s*\('
    )
    
    # Process only diff lines that start with '+' or '-'
    diff_lines = [line for line in change.splitlines() if line.startswith('-')]
    
    for line in diff_lines:
        # Fully qualified calls: add "::" + captured function name.
        for match in pattern_fq.finditer(line):
            func = match.group(1)
            API_set.add("::" + func + "(")
        # Dot-based calls:
        for match in pattern_dot.finditer(line):
            API_set.add(match.group(1) + "(")
        # Arrow-based calls:
        for match in pattern_arrow.finditer(line):
            API_set.add(match.group(1) + "(")
        # Unqualified calls:
        for match in pattern_unq.finditer(line):
            func = match.group(2)
            API_set.add(func + "(")
    
    return list(API_set)


In [ ]:
# def get_APIs(change):
#     """
#     Extracts both C++ member/namespace API calls and C-style function calls from a code change.
#     Returns a list of unique API call substrings.
#     """
#     # Pattern for C++ style: matches calls preceded by ->, . or ::
#     pattern_cpp = r"(?:->|\.|::)[a-zA-Z0-9_]+\("
#     # Pattern for C-style: matches function calls that are not necessarily preceded by any operator.
#     pattern_c = r"\b[a-zA-Z_][a-zA-Z0-9_]*\("

#     # Find matches
#     apis_cpp = re.findall(pattern_cpp, change)
#     apis_c = re.findall(pattern_c, change)
    
#     # Optionally remove the ones already captured by pattern_cpp from pattern_c to avoid duplicates.
#     # This is one simple way to ensure that calls captured with operators are not repeated.
#     apis_c = [api for api in apis_c if not any(api in cpp_api for cpp_api in apis_cpp)]
    
#     # Combine results and remove duplicates
#     all_apis = set(apis_cpp + apis_c)
#     return list(all_apis)

In [ ]:
# def get_APIs(change):
#     API_list = []
#     API = re.findall(r"\.[a-zA-Z0-9_]+\(", change)
#     if len(API) > 0:
#         API_list = list(set(API))
#     return API_list

In [25]:
# why 999? Read the paper
def get_similar_example_id(code_before, top_k=999):
    results = collection.query(
        query_texts=code_before,
        n_results=top_k,
    )
    return results["ids"]

In [26]:
input_path = "/home/akazad/test_tool_perfapim/perfapim/dataset/stratified_splits/ctx_t3_fold_5_test.jsonl" # original test set
example_path = "/home/akazad/test_tool_perfapim/llmapitool/my_tool_exp/data/cpp_fix_rules_stratified_ctxt3f5.json" # testing 50 DB samples
output_path = "result/cpp_detection_result_stratified_ctx_t3_f5.json" # test results

In [27]:
#this is just to get the value of n, to extract APIs directly from the dataset
import json
#path = "/home/akazad/test_tool_perfapim/perfapim/dataset/stratified_splits/ctx_t4_fold_1_train.jsonl"
path = "/home/akazad/test_tool_perfapim/perfapim/dataset/output_794_context_enhanced_merged_latest.jsonl"
data_dict = {}
with open(path, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue
        data_dict[record["number"]] = record

In [29]:
def extract_function_line(input_text: str, func_name: str) -> str:
    """
    Splits the input text by newline and searches for the first line that 
    contains the function name followed by '('.
    
    Parameters:
        input_text (str): The newline-separated string to search.
        func_name (str): The name of the function to search for (e.g., "mempcpy").
    
    Returns:
        str: The first matching line containing the function call, or an empty string if no match is found.
    """
    func_name = func_name.rstrip("(")
    # Create the search pattern (e.g., "mempcpy(")
    search_pattern = f"{func_name}"
    
    # Split the text into lines
    lines = input_text.splitlines()
    print("func:", func_name)
    
    # Loop over each line and check if it contains the search pattern
    for line in lines:
        if search_pattern in line:
            return line
    
    # Return an empty string if no match is found
    return ""


In [ ]:
# Example usage:
input_text = """- api_context: void* mempcpy(void* dest, const void* src, size_t n);, library: 'Standard C++ (part of the C standard library)'
- api_context: void* memcpy(void* dest, const void* src, size_t n);, library: 'Standard C++ (part of the C standard library)'"""

matched_line = extract_function_line(input_text, "mempcpy(")
print("Matched Line:", matched_line)

In [248]:
print(extract_function_line(data_dict[1]['enhanced_context'], "memcpy("))

func: memcpy
- api_context: void* memcpy(void* dest, const void* src, size_t n);, library: 'Standard C++ (part of the C standard library)'


In [30]:
print(len(data_dict))

697


In [31]:
def parse_fix_pattern(response: str) -> str:
    pattern_key = "<pattern>:"
    lower_response = response.lower()
    index = lower_response.find(pattern_key.lower())

    # Case 1: <pattern>: is found
    if index != -1:
        start_idx = index + len(pattern_key)
        extracted = response[start_idx:].strip()
        if extracted.lower() == "na":
            return "NA"
        return extracted

    # Case 2: <pattern>: is not found
    stripped_response = response.strip()
    if stripped_response.lower() == "na":
        return "NA"
    return stripped_response

In [32]:
# read the database
example_dict = {}
with open(example_path, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        #example_dict[item["number"]] = item
    #data = json.loads(f.read())
    #for line in data:
        pattern = parse_fix_pattern(record["fix_pattern"])
        #APIs = get_APIs(record["change"])
        n = record["number"]
        APIs = data_dict[n]["API"]
        print(APIs)
        item = {
            "number": record["number"],
            "change": "".join(record["change"]),
            "fix_pattern": pattern,
            "APIs": APIs,
        }
        print(pattern)
        example_dict[record["number"]] = item
print(len(example_dict.keys()))

['memcpy(', '->Size(', 'mempcpy(']
if the code uses mempcpy unnecessarily when memcpy suffices, replace it with memcpy for improved performance since memcpy is typically optimized for this use case and avoids potential unnecessary complications.
['rand(', '->Uniform(']
if a call to gRandom->Uniform is replaced with a direct use of rand() from the Standard C Library, consider if this change is made to avoid the overhead or dependency of the ROOT Framework for simpler random number generation.
['::Open(', 'strstr(', '::SetCacheFileDir(']
if a filename is detected as a URL starting with "http://", utilize the "CACHEREAD" option when opening the file through the TFile API to potentially improve file access performance by caching the contents locally.
['strlen(', 'strncat(', 'strlcpy(', 'strlcat(', 'strncpy(']
if string manipulation is done using strncpy and strncat, replace them with strlcpy and strlcat for improved safety and performance, as strlcpy and strlcat not only copy and concatena

In [ ]:
#input_path = "/home/akazad/test_tool_perfapim/perfapim/dataset/perf_api_misuse_cpp_final_test.jsonl" # original test set

Here I need to plugin the SNIPPET (Step 2)

In [34]:
def extract_code_changes_simple(diff_text):
    """
    A simplified version that takes a string of GitHub diff lines (for a single function),
    splits into lines, and classifies them as added, removed, or context/hunk header.
    
    If a line starts with '@@', we only keep the trailing code (e.g. 'def tensordot(') 
    and store it as context for both code_before_change and code_after_change.
    """

    # Split diff into lines
    change_lines = diff_text.split('\n')

    # Initialize empty containers for each category
    change = ""
    context = ""
    code_before_change = ""
    code_after_change = ""
    added_code = ""
    removed_code = ""

    # Regex to match hunk header: @@ -xxx,xxx +xxx,xxx @@
    # We'll capture everything after the second '@@'
    hunk_header_pattern = re.compile(r'^@@.*@@(.*)')

    for line in change_lines:
        # Always store the full line in 'change' (raw diff)
        change += line + "\n"

        if line.startswith("@@"):
            # Attempt to extract trailing code after second '@@'
            match = hunk_header_pattern.match(line)
            if match:
                # e.g. if line is "@@ -393,7 +393,7 @@ def tensordot("
                # match.group(1) => " def tensordot("
                trailing_code = match.group(1).strip()
                if trailing_code:
                    # Store that trailing code as context
                    context += trailing_code + "\n"
                    code_before_change += trailing_code + "\n"
                    code_after_change += trailing_code + "\n"
                # If there's no trailing code, we don't add anything
            # Move on to next line, skipping +, -, or else checks
            continue

        elif line.startswith("+"):
            # Lines added in the patch
            content = line[1:]  # remove leading '+'
            code_after_change += content + "\n"
            added_code += content + "\n"

        elif line.startswith("-"):
            # Lines removed in the patch
            content = line[1:]  # remove leading '-'
            code_before_change += content + "\n"
            removed_code += content + "\n"

        else:
            # Everything else = unchanged context
            context += line + "\n"
            code_before_change += line + "\n"
            code_after_change += line + "\n"

    return {
        "added_code": added_code.strip(),
        "removed_code": removed_code.strip(),
        "context": context.strip(),
        "code_before_change": code_before_change.strip(),
        "code_after_change": code_after_change.strip(),
        "change": change.strip()
    }


In [35]:
import json

# 291 test samples
test_data_dict = {}
 # i am reading a .jsonl file (test.jsonl) and storing the data in a dictionary
with open(input_path, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        change = "".join(record["change"])
        #APIs = get_APIs(change)
        APIs = record["API"] # directly getting the APIs from the dataset
        n = record["number"]
        #fix the context here
        parsed_result = extract_code_changes_simple(record["code_diff_x"])
        snippet = parsed_result["code_before_change"]
        print(f"N: {n}, S:\n {snippet}")
        print(f"APIs: {APIs}")
        
        api_before = get_APIs_cpp(record["code_diff_x"])
        print(f"api_before: {api_before}")
        item = {
            "number": record["number"],
            "API": APIs,
            "API_before": api_before,
            #"context": record["change"],  # need to fix here, context should be the code before the change
            "context": snippet
            # add additoanl context here
        }
        test_data_dict[item["number"]] = item

# Convert dictionary to a list
data = list(test_data_dict.values())

print(len(data))
print(data[0].keys())

N: 5, S:
 Int_t TGX11::SetTextFont(char *fontname, ETextSetMode mode)
          XSetFont(fDisplay, *gGCtext, gTextFont->fid);
          XSetFont(fDisplay, *gGCinvt, gTextFont->fid);
          gFont[gCurrentFontNumber].id = gTextFont;
         strcpy(gFont[gCurrentFontNumber].name,fontname);
          gCurrentFontNumber++;
          if (gCurrentFontNumber == kMAXFONT) gCurrentFontNumber = 0;
       }
APIs: ['strcpy(', 'strlcpy(']
api_before: ['strcpy(']
N: 6, S:
 Bool_t TXMLPlayer::ProduceSTLstreamer(ostream& fs, TClass* cl, TStreamerSTL* el,
       strlcpy(tabs2, tabs, sizeof(tabs2));
 
       if (isptr) {
         strncat(tabs2, tab1, sizeof(tabs2));
          fs << tabs << "if (" << (isarr ? "*cont" : "cont") << "==0) {" << endl;
          fs << tabs2 << "buf.WriteSTLsize(0" << (isstr ? ",true);" : ");") << endl;
          fs << tabs << "} else {" << endl;
APIs: ['strncat(', 'strlcat(']
api_before: ['strncat(']
N: 32, S:
 // zlib compress2 function resulted in a 40+% reduction versus

In [ ]:
# #291 test samples
# data_dict = {}
# with open(input_path, encoding="utf-8") as f:
#     data = json.loads(f.read())
#     for line in data:
#         change = "".join(line["change"])
#         APIs = get_APIs(change)
#         item = {
#             "number": line["number"],
#             "API": APIs,
#             "context": line["change"],
#         }
#         data_dict[item["number"]] = item

# data = []
# for key in data_dict.keys():
#     data.append(data_dict[key])

# print(len(data))
# print(data[0].keys())

# prompt templates

In [36]:
template_1 = """
As an experienced software developer, you have a strong ability to read and understand code snippet. If you encounter a question to which you don't know the answer, you acknowledge your lack of knowledge.

Task:
Describe what the following code snippet does in two sentences.

Code snippet:
{code_before}

Additional code snippet context (if available):
{additional_context}

Answer:
"""

template_2 = """

Please read the following code snippet and API misuse fix rules. Then, answer if the fix patterns can be applied to the code snippet.
If any of the patterns can be applied, answer "Yes",  if not, answer "No" directly.

Code snippet:
{code_before}

Additional code snippet context (if available):
{additional_context}

Fix rules:
{fix_rules}

Decision:(Yes/No)
"""

In [255]:
len(data)

88

In [ ]:
#output_path = "result/cpp_detection_result_149_s3_t2.json" # test results

In [37]:
print(data[0]['context'])

Int_t TGX11::SetTextFont(char *fontname, ETextSetMode mode)
          XSetFont(fDisplay, *gGCtext, gTextFont->fid);
          XSetFont(fDisplay, *gGCinvt, gTextFont->fid);
          gFont[gCurrentFontNumber].id = gTextFont;
         strcpy(gFont[gCurrentFontNumber].name,fontname);
          gCurrentFontNumber++;
          if (gCurrentFontNumber == kMAXFONT) gCurrentFontNumber = 0;
       }


START OF TESTING LOOP

In [38]:
# actual testing loop for 291

# 14
example_len_list = []
for i in range(0, len(data)): # for each datapoints the do this
    # for i in range(0, 2):
    print("current_index:", i, "/", len(data))
    #if i >= 10: break
    code_before = ""
    #for x in data[i]["context"]:
    #    code_before += x + "\n"  # each line is being added to the code_before variable
    # I should get it from the dictionary I stored earlier step
    code_before = data[i]["context"] # my context is the code before the change prepapred before
     # the change is the code diff
    number = data[i]["number"]
    change = data_dict[number]["change"]
    print("number", number)
    if code_before == "":
        continue

    query_APIs = data[i]["API"]   # problematic APIs
    query_APIs_only = data[i]['API_before'] # get the APIs from the code before the change
    print("query_API_only", query_APIs_only)
    enhanced_context = data_dict[number]["enhanced_context"] # get the context from the code before the change
    
    lines = []
    for x in query_APIs_only:
        t = extract_function_line(enhanced_context, x)
        lines.append(t)
    line = "\n".join(lines)
    print("line", line)
    
    #print("query_APIs_only_query", query_APIs_only_query)

    prompt_1 = template_1.format(code_before=code_before, additional_context = line) # explain the given input code
    prompt_1 = prompt_1.replace("\n\n", "\n")


    # call chatgpt to explain the code
            # apply llm
    explain = completion_with_backoff(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt_1}
        ]
    )
    explain = explain.choices[0].message.content
    
    # explain = completion_with_backoff(
    #     model="gpt-3.5-turbo",
    #     messages=[
    #         {"role": "user", "content": prompt_1} # prompt to explain the code
    #     ]
    # )
    # explain = explain["choices"][0]["message"]["content"]

    # get similar example using chromadb, given the explaination of the current input code
    sorted_example_ids = get_similar_example_id(explain) # call to vectordb to get the similar examples to be used in the prompt
    sorted_example_ids = [int(x) for x in sorted_example_ids[0]]
    print("sorted_example_ids: ", sorted_example_ids)
    example_list_filtered = []
    
    # the following loop is to filter the examples (4) that contain the query APIs
    for id in sorted_example_ids:
        #if id>=3573:
            # the 600th example is at id =3573, we reserve 891-600=291 examples for verification
        #    continue
        if len(example_list_filtered) >= top_k: # loop should break if we have enough examples (4)
            break
        # retrieve the example from the example_dict
        item = example_dict[id] # now I understand why we need the example_dict created earlier
        
        example_APIs = item["APIs"] # extract the APIs from the example
        
        for single_API in query_APIs: # loop through the query APIs
            if single_API in example_APIs: # if the query API is in the example API
                example_list_filtered.append(item) # add the example to the filtered list

    print("example_list_filtered: ",len(example_list_filtered))
    fix_pattern_prompt = ""
    idx = 0
    # ignore if no example found
    if len(example_list_filtered) == 0:
        continue
    
    # create the prompt for the second question
    # creates list of fix patterns, max 20
    for x in example_list_filtered:
        if idx >= 20:
            continue
        idx += 1
        fix_pattern = x["fix_pattern"] # fix pattern from the i'th example
        fix_pattern_prompt += "Fix pattern " + str(idx) + " :\n" # building the prompt
        fix_pattern_prompt += fix_pattern + "\n"

    prompt_2 = template_2.format(
        code_before=code_before, fix_rules=fix_pattern_prompt, additional_context = line) # supply the code and the fix patterns list
    prompt_2 = prompt_2.replace("\n\n", "\n")
    print("prompt_2", len(prompt_2))


    # now call chatgpt to get the "Yes" or "No" answer
    # decition = openai.ChatCompletion.create(
    decition = completion_with_backoff(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt_2}
        ]
    )
    decition = decition.choices[0].message.content

    output = {
        "number": number,
        "code_before": code_before,
        "code explain":explain,
        "example": fix_pattern_prompt,
        "detection_result": decition,
         "prompt_2": prompt_2,
    }

    with open(output_path, 'a') as f:
        json.dump(output, f)
        f.write(os.linesep)

current_index: 0 / 67
number 5
query_API_only ['strcpy(']
func: strcpy
line - api_context: char *strcpy(char *dest, const char *src), library: 'Standard C library (part of <string.h>)'


Number of requested results 999 is greater than number of elements in index 269, updating n_results = 269


sorted_example_ids:  [128, 863, 482, 639, 780, 942, 694, 593, 308, 931, 1086, 716, 679, 945, 922, 1000, 685, 99, 912, 575, 1093, 973, 78, 349, 610, 73, 476, 142, 143, 840, 865, 547, 232, 16, 544, 1010, 181, 778, 310, 156, 997, 215, 201, 72, 8, 475, 743, 807, 1041, 190, 695, 40, 3, 1052, 454, 174, 963, 803, 798, 657, 799, 715, 354, 984, 562, 546, 595, 543, 218, 554, 204, 146, 632, 524, 939, 529, 41, 65, 989, 648, 242, 4, 407, 385, 986, 631, 624, 70, 84, 680, 1110, 173, 590, 959, 255, 1089, 452, 39, 219, 412, 1029, 1, 118, 581, 36, 844, 416, 521, 318, 198, 630, 1064, 292, 214, 712, 225, 751, 847, 1040, 602, 494, 13, 812, 344, 1119, 904, 949, 806, 398, 927, 650, 372, 538, 321, 725, 905, 438, 131, 1082, 722, 402, 979, 1043, 619, 138, 830, 1078, 456, 253, 427, 1056, 382, 607, 564, 15, 707, 54, 616, 205, 509, 148, 288, 579, 447, 988, 573, 94, 541, 998, 585, 286, 1092, 749, 371, 102, 1020, 1115, 343, 108, 513, 180, 324, 701, 367, 417, 291, 583, 613, 261, 352, 228, 971, 74, 885, 409, 572, 653,

Number of requested results 999 is greater than number of elements in index 269, updating n_results = 269


sorted_example_ids:  [513, 610, 482, 349, 173, 365, 118, 402, 308, 286, 562, 650, 181, 352, 541, 201, 475, 840, 214, 251, 198, 372, 371, 922, 639, 65, 452, 131, 255, 716, 712, 547, 575, 334, 41, 204, 1040, 619, 78, 865, 584, 702, 146, 543, 321, 725, 1021, 679, 344, 695, 579, 1002, 476, 583, 694, 912, 593, 94, 1052, 430, 630, 218, 219, 984, 73, 613, 142, 29, 885, 986, 99, 242, 343, 1, 156, 544, 632, 863, 554, 292, 799, 84, 407, 143, 40, 715, 102, 416, 36, 8, 409, 581, 945, 335, 310, 778, 232, 454, 354, 120, 108, 680, 631, 318, 128, 616, 1089, 939, 205, 989, 648, 106, 624, 838, 368, 529, 311, 847, 291, 190, 70, 1086, 138, 521, 3, 288, 174, 16, 998, 768, 550, 524, 595, 657, 367, 566, 653, 215, 228, 602, 53, 988, 1078, 412, 707, 540, 438, 72, 39, 67, 709, 385, 383, 1029, 274, 743, 546, 954, 107, 573, 742, 959, 225, 835, 904, 587, 346, 474, 830, 590, 394, 969, 963, 905, 1119, 711, 417, 992, 1115, 180, 949, 1110, 24, 1041, 701, 765, 798, 722, 62, 382, 256, 538, 258, 13, 427, 64, 914, 572, 75

...

sorted_example_ids:  [997, 1086, 1010, 942, 973, 984, 482, 807, 78, 931, 575, 922, 863, 72, 1000, 840, 610, 8, 912, 1082, 716, 349, 844, 1043, 780, 544, 128, 407, 344, 806, 204, 308, 1093, 798, 156, 174, 1110, 181, 998, 142, 593, 904, 679, 639, 945, 201, 53, 632, 143, 371, 547, 657, 310, 173, 190, 36, 581, 546, 543, 1041, 631, 885, 385, 695, 476, 712, 438, 748, 16, 524, 255, 402, 694, 454, 989, 562, 865, 475, 214, 541, 354, 554, 905, 99, 218, 372, 416, 742, 364, 1, 579, 242, 939, 550, 529, 292, 84, 564, 219, 70, 685, 715, 41, 215, 94, 722, 494, 474, 616, 321, 286, 1089, 228, 630, 959, 198, 205, 587, 73, 743, 3, 225, 566, 409, 118, 131, 680, 538, 447, 102, 613, 65, 258, 624, 108, 607, 927, 949, 847, 382, 573, 452, 653, 383, 584, 648, 1029, 768, 1103, 4, 799, 583, 963, 427, 256, 352, 509, 107, 619, 650, 590, 291, 1115, 711, 288, 412, 602, 232, 148, 367, 572, 15, 585, 1021, 251, 311, 1078, 595, 343, 54, 707, 725, 513, 1052, 180, 521, 40, 146, 838, 1040, 368, 677, 365, 13, 261, 812, 253, 7

Number of requested results 999 is greater than number of elements in index 269, updating n_results = 269


sorted_example_ids:  [1082, 1043, 973, 942, 931, 1093, 807, 1000, 1086, 984, 482, 310, 998, 997, 885, 657, 546, 412, 407, 653, 190, 292, 922, 863, 844, 476, 905, 513, 806, 4, 610, 174, 36, 349, 590, 1110, 214, 945, 581, 72, 780, 402, 344, 631, 722, 904, 251, 1, 632, 232, 742, 712, 102, 354, 94, 619, 409, 949, 695, 595, 509, 630, 685, 385, 308, 613, 798, 372, 743, 541, 939, 371, 219, 367, 73, 475, 840, 716, 131, 156, 575, 1041, 84, 554, 173, 228, 494, 181, 579, 1040, 1103, 584, 927, 438, 1010, 15, 41, 29, 583, 616, 53, 78, 564, 447, 321, 550, 838, 562, 593, 39, 602, 142, 589, 74, 368, 454, 650, 707, 702, 474, 128, 352, 218, 680, 573, 524, 201, 847, 255, 215, 382, 13, 242, 648, 624, 204, 3, 65, 416, 585, 225, 677, 572, 505, 335, 383, 865, 566, 8, 346, 812, 768, 694, 108, 198, 205, 452, 989, 751, 64, 261, 258, 118, 748, 979, 136, 709, 912, 1078, 107, 543, 364, 106, 253, 180, 679, 715, 529, 334, 67, 799, 1029, 587, 417, 720, 544, 639, 256, 1020, 547, 143, 1064, 430, 62, 394, 959, 711, 311,

Number of requested results 999 is greater than number of elements in index 269, updating n_results = 269


sorted_example_ids:  [505, 311, 778, 1041, 321, 904, 174, 632, 840, 1052, 94, 679, 631, 1021, 838, 686, 922, 742, 180, 476, 494, 905, 680, 308, 1020, 979, 639, 382, 575, 578, 657, 585, 454, 430, 368, 610, 725, 409, 344, 102, 598, 288, 524, 529, 292, 630, 39, 1, 198, 201, 722, 716, 78, 72, 349, 371, 310, 998, 57, 695, 416, 1078, 581, 912, 554, 232, 274, 572, 945, 885, 653, 513, 190, 354, 865, 989, 407, 70, 84, 367, 138, 215, 969, 1086, 648, 59, 131, 709, 573, 830, 335, 324, 986, 509, 1089, 712, 13, 984, 1119, 694, 616, 624, 173, 863, 181, 963, 343, 780, 3, 447, 949, 385, 29, 798, 107, 118, 973, 685, 812, 939, 214, 707, 440, 54, 318, 228, 715, 751, 204, 619, 53, 720, 8, 225, 1029, 790, 106, 743, 589, 108, 253, 847, 772, 136, 156, 749, 844, 562, 541, 942, 547, 914, 475, 711, 148, 242, 417, 261, 286, 566, 402, 74, 521, 954, 799, 795, 16, 806, 4, 474, 803, 1040, 579, 346, 64, 372, 593, 959, 590, 128, 927, 1000, 1082, 613, 1093, 544, 1103, 835, 701, 24, 352, 41, 607, 73, 1043, 1028, 452, 255

In [69]:
#print current directory
print(os.getcwd())

/home/akazad/test_tool_perfapim/llmapitool/my_tool_exp


In [19]:
print(output_path)

result/cpp_detection_result_stratified_t3f1.json
